sources:

https://graphframes.github.io/graphframes/docs/_site/user-guide.html

https://docs.databricks.com/_static/notebooks/graphframes-user-guide-py.html

In [34]:
import os
import pandas as pd
import numpy as np

import findspark
findspark.init()

from pyspark import *
from pyspark.sql import *
from graphframes import *

In [2]:
# https://graphframes.github.io/graphframes/docs/_site/quick-start.html
# https://stackoverflow.com/questions/65011599/how-to-start-graphframes-on-spark-on-pyspark-on-juypter-on-docker
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages graphframes:graphframes:0.8.1-spark3.0-s_2.12 pyspark-shell'

In [3]:
# Start Spark context
spark = SparkSession.builder.appName('graphx-network-analysis').getOrCreate()

21/10/19 21:48:49 WARN Utils: Your hostname, mark-machine resolves to a loopback address: 127.0.1.1; using 192.168.0.102 instead (on interface wlp8s0)
21/10/19 21:48:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/mark/.ivy2/cache
The jars for the packages stored in: /home/mark/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4e37376d-3dd9-4549-b252-c9d8ce911acb;1.0
	confs: [default]
	found graphframes#graphframes;0.8.1-spark3.0-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 109ms :: artifacts dl 3ms
	:: modules in use:
	graphframes#graphframes;0.8.1-spark3.0-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-------------------------------------

In [4]:
!mkdir -p data
!wget http://snap.stanford.edu/data/amazon0302.txt.gz -P data/

--2021-10-19 21:48:52--  http://snap.stanford.edu/data/amazon0302.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4664334 (4,4M) [application/x-gzip]
Saving to: ‘data/amazon0302.txt.gz.1’

amazon0302.txt.gz.1 100%[===================>]   4,45M   417KB/s    in 11s     

2021-10-19 21:49:04 (401 KB/s) - ‘data/amazon0302.txt.gz.1’ saved [4664334/4664334]



In [5]:
!gzip -dkfv data/amazon0302.txt.gz

data/amazon0302.txt.gz:	 73.0% -- created data/amazon0302.txt


In [93]:
def read_data(from_file_name, exclude_meta=True):
    df_edges = pd.read_csv(from_file_name, sep="\t", header=None, skiprows=4, names=["src", "dst"])
    vertices_np = \
        np.unique(np.hstack([df_edges["src"].unique(), df_edges["dst"].unique()]))
    
    df_vertices = pd.DataFrame(data={"id": vertices_np})
    
    meta_cols = ["ansi", "title", "group", "salesrank", "similar", "categories", "reviews"]
    
    if exclude_meta:
        df_vertices[meta_cols] = np.nan
    else:
        # read meta file and construct column values
        # until then same as no meta
        df_vertices[meta_cols] = np.nan

    
    return df_edges, df_vertices
    
from_file_name = "data/amazon0302.txt"
df_edges, df_vertices = read_data(from_file_name=from_file_name)

In [94]:
print(df_edges)
print(df_vertices)

            src     dst
0             0       1
1             0       2
2             0       3
3             0       4
4             0       5
...         ...     ...
1234872  262110  262099
1234873  262110  262100
1234874  262110  262101
1234875  262110  262106
1234876  262110  262107

[1234877 rows x 2 columns]
            id  ansi  title  group  salesrank  similar  categories  reviews
0            0   NaN    NaN    NaN        NaN      NaN         NaN      NaN
1            1   NaN    NaN    NaN        NaN      NaN         NaN      NaN
2            2   NaN    NaN    NaN        NaN      NaN         NaN      NaN
3            3   NaN    NaN    NaN        NaN      NaN         NaN      NaN
4            4   NaN    NaN    NaN        NaN      NaN         NaN      NaN
...        ...   ...    ...    ...        ...      ...         ...      ...
262106  262106   NaN    NaN    NaN        NaN      NaN         NaN      NaN
262107  262107   NaN    NaN    NaN        NaN      NaN         NaN      NaN


In [95]:
vertices = spark.createDataFrame(df_vertices, list(df_vertices.columns.values))
edges = spark.createDataFrame(df_edges, list(df_edges.columns.values))
g = GraphFrame(vertices, edges)

In [96]:
g.vertices.show()
g.edges.show()

+---+----+-----+-----+---------+-------+----------+-------+
| id|ansi|title|group|salesrank|similar|categories|reviews|
+---+----+-----+-----+---------+-------+----------+-------+
|  0| NaN|  NaN|  NaN|      NaN|    NaN|       NaN|    NaN|
|  1| NaN|  NaN|  NaN|      NaN|    NaN|       NaN|    NaN|
|  2| NaN|  NaN|  NaN|      NaN|    NaN|       NaN|    NaN|
|  3| NaN|  NaN|  NaN|      NaN|    NaN|       NaN|    NaN|
|  4| NaN|  NaN|  NaN|      NaN|    NaN|       NaN|    NaN|
|  5| NaN|  NaN|  NaN|      NaN|    NaN|       NaN|    NaN|
|  6| NaN|  NaN|  NaN|      NaN|    NaN|       NaN|    NaN|
|  7| NaN|  NaN|  NaN|      NaN|    NaN|       NaN|    NaN|
|  8| NaN|  NaN|  NaN|      NaN|    NaN|       NaN|    NaN|
|  9| NaN|  NaN|  NaN|      NaN|    NaN|       NaN|    NaN|
| 10| NaN|  NaN|  NaN|      NaN|    NaN|       NaN|    NaN|
| 11| NaN|  NaN|  NaN|      NaN|    NaN|       NaN|    NaN|
| 12| NaN|  NaN|  NaN|      NaN|    NaN|       NaN|    NaN|
| 13| NaN|  NaN|  NaN|      NaN|    NaN|

21/10/19 22:44:48 WARN TaskSetManager: Stage 50 contains a task of very large size (2213 KiB). The maximum recommended task size is 1000 KiB.
21/10/19 22:44:48 WARN TaskSetManager: Stage 51 contains a task of very large size (1204 KiB). The maximum recommended task size is 1000 KiB.


In [97]:
print((g.vertices.count(), len(g.vertices.columns)))
print((g.edges.count(), len(g.edges.columns)))

21/10/19 22:44:55 WARN TaskSetManager: Stage 52 contains a task of very large size (2213 KiB). The maximum recommended task size is 1000 KiB.


(262111, 8)


21/10/19 22:44:56 WARN TaskSetManager: Stage 54 contains a task of very large size (1204 KiB). The maximum recommended task size is 1000 KiB.


(1234877, 2)


In [98]:
g.degrees.show()
g.inDegrees.show()
g.outDegrees.show()

21/10/19 22:45:05 WARN TaskSetManager: Stage 56 contains a task of very large size (1204 KiB). The maximum recommended task size is 1000 KiB.
21/10/19 22:45:07 WARN TaskSetManager: Stage 58 contains a task of very large size (1204 KiB). The maximum recommended task size is 1000 KiB.


+----+------+
|  id|degree|
+----+------+
|  26|    17|
|  29|    45|
| 474|     4|
|1677|    13|
|1697|     7|
| 964|    13|
|1806|    18|
|1950|     9|
|2453|     8|
|3091|     8|
|2040|     6|
|3506|    15|
|2214|    24|
|2250|     9|
|5385|     9|
|7225|    13|
|2509|     6|
|5409|    14|
|2529|    18|
|2927|     7|
+----+------+
only showing top 20 rows

+----+--------+
|  id|inDegree|
+----+--------+
|  26|      12|
|  29|      40|
| 474|       4|
|1677|       8|
|1697|       2|
| 964|       8|
|1806|      13|
|1950|       4|
|2453|       3|
|3091|       3|
|2040|       1|
|3506|      10|
|2214|      19|
|2250|       4|
|5385|       4|
|7225|       8|
|2509|       1|
|5409|       9|
|2529|      13|
|2927|       2|
+----+--------+
only showing top 20 rows



21/10/19 22:45:07 WARN TaskSetManager: Stage 60 contains a task of very large size (1204 KiB). The maximum recommended task size is 1000 KiB.


+----+---------+
|  id|outDegree|
+----+---------+
|  26|        5|
|  29|        5|
| 964|        5|
|1677|        5|
|1697|        5|
|1806|        5|
|1950|        5|
|2040|        5|
|2214|        5|
|2250|        5|
|2453|        5|
|2509|        5|
|2529|        5|
|2927|        5|
|3091|        5|
|3506|        5|
|3764|        5|
|4590|        5|
|4823|        5|
|4894|        5|
+----+---------+
only showing top 20 rows

